# ex-15 Vectorize - Extract Raster Boundary to GeoDataFrame

In the current tutorial, we will apply the vectorize function from the python package of geocude to extract boundary of a DEM data.

In [ ]:
"""
Module for vector methods
https://corteva.github.io/geocube/stable/_modules/geocube/vector.html#vectorize
"""
import warnings

import geopandas
import numpy
import rasterio.features
import rioxarray  # noqa: F401 pylint: disable=unused-import
import shapely.geometry
import xarray


def vectorize(data_array: xarray.DataArray) -> geopandas.GeoDataFrame:
    """
    .. versionadded:: 0.4.0

    Powered by: :func:`rasterio.features.shapes`

    Convert 2D :class:`xarray.DataArray` into
    a :class:`geopandas.GeoDataFrame`.

    The ``nodata``,  ``CRS``, and ``transform`` of the :class:`xarray.DataArray`
    are determined using ``rioxarray``.

    Helpful references:

    - https://corteva.github.io/rioxarray/stable/getting_started/crs_management.html
    - https://corteva.github.io/rioxarray/stable/getting_started/nodata_management.html
    - https://gis.stackexchange.com/questions/379412/creating-geopandas-geodataframe-from-rasterio-features


    Parameters
    ----------
    data_array: xarray.DataArray
        Input 2D DataArray raster.

    Returns
    -------
    geopandas.GeoDataFrame
    """
    # nodata mask
    mask = None
    if data_array.rio.nodata is not None:
        if numpy.isnan(data_array.rio.nodata):
            mask = ~data_array.isnull()
        else:
            mask = data_array != data_array.rio.nodata

    # vectorize generator
    vectorized_data = (
        (value, shapely.geometry.shape(polygon))
        for polygon, value in rasterio.features.shapes(
            data_array,
            transform=data_array.rio.transform(),
            mask=mask,
        )
    )

    if data_array.name:
        name = data_array.name
    else:
        warnings.warn("The array has no name. Column name defaults to _data")
        name = "_data"

    return geopandas.GeoDataFrame(
        vectorized_data,
        columns=[name, "geometry"],
        crs=data_array.rio.crs,
    )

## Read data

In [ ]:
infile = Path(r"data/DEM_XX.tif")
da = rx.open_rasterio(infile, mask_and_scale=True, chunks={"x": 512, 'y': 512})

## Process

- convert valid DEM values to 0.0
- keep other values as np.nan

In [ ]:
da = xr.where(np.isnan(da), np.nan, 0.0).astype('float32')
da.name = 'DEM'

## Vectorize to extract boundry

poly = vectorize(da).dropna()

# save 
outfile = Path(f"{infile.stem}.shp")
poly.to_file(outfile)

In [ ]:
poly

## Have a simple visualization

In [ ]:
poly.plot()

## References and Resources

https://corteva.github.io/geocube/stable/examples/vectorize.html

https://docs.xarray.dev/en/stable/

https://corteva.github.io/rioxarray/html/rioxarray.html

https://rasterio.readthedocs.io/en/stable/